# Stem Analysis

This notebook provides code to analyse and explore point cloud tree stems. As input you can either use an already separated stem point cloud or use a complete tree and use the provided separation code (see `option B` in step 1)

---------------

##### Imports

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import os
import trimesh
import numpy as np
import open3d as o3d

from utils import (
      ahn_utils,
      las_utils,
      tree_utils,
      o3d_utils
  )
from utils.interpolation import FastGridInterpolator
from misc.fitcyclinders import fit_vertical_cylinder_3D, show_cylinders

from labels import Labels

### 1. Load Data
---

**Option A**: Load stem point cloud

In [ ]:
las_file = '../../dataset/sonarski/stem_121913_487434.las'
stem_cloud = o3d_utils.read_las(las_file)

**Option B**: Load tree point cloud and separate using `src/utils/tree_utils.py`

In [ ]:
adTree_exe = '../../AdTree/build/bin/AdTree.app/Contents/MacOS/AdTree'

# Load point cloud data
tree_cloud = o3d_utils.read_las('../../dataset/cyclo/filtered_tree_121913_487434.las')

# Separate stem from tree
stem_cloud, _ = tree_utils.tree_separate(tree_cloud, adTree_exe)

In [ ]:
o3d.visualization.draw_geometries([stem_cloud])

### 2. Stem Analysis
---

**Fit cyclinders to stem**

In [ ]:
stem_cylinders = tree_utils.fit_cylinders_to_stem(stem_cloud, .25)

In [ ]:
# Visualise stem fit
show_cylinders(stem_cylinders, resolution=25, cloud=stem_cloud)

In [ ]:
mesh = o3d_utils.mesh_from_cylinders(stem_cylinders, tree_utils.tree_colors['stem'])
o3d_utils.plot_mesh(mesh)

**Circumerferential Completness Index (CCI)**

CCI is simply the fraction of a circle with point coverage in a stem slice as illustrated below. This provides an indication of how complete your stem coverage is. In a single scan TLS point cloud, you cannot get a CCI greater than 0.5 (assuming the cylinder fitting was not erroneous), as only one side of the tree is mapped. If you have completely scanned the tree (at the measurement location), you should get a CCI of 1.0 (the highest possible CCI).

![CCI.jpg](../../imgs/CCI.jpg)

The figure is from this paper: https://doi.org/10.3390/rs12101652 if you would like a more detailed explanation of the idea.

In [ ]:
stem_CCI = (np.min(stem_cylinders[:,4]), np.max(stem_cylinders[:,4]))
print(f" The minimal and maximal CCI of the scanned stem is {stem_CCI}")

`Diameter at breast height` == 1.3 m

**Stem Angle**

In [ ]:
stem_angle = tree_utils.stem_angle(stem_cylinders)
print(f"Stem angle is {stem_angle:.2f} degrees")

**Stem Angle Direciton**

In [ ]:
# Angle to witch the stem is falling towards
compas_bearing = tree_utils.stem_bearing(stem_cylinders)
print(f"Stem bearing is {compas_bearing:.2f} degrees")

**Stem Location**

In [ ]:
# Load AHN Surface data
ahn_data_folder = '../../dataset/ahn_surf/'
npz_reader = ahn_utils.NPZReader(ahn_data_folder)
treecode = las_utils.get_treecode_from_filename(las_file)
ground_cloud = npz_reader.get_surface(treecode)

In [ ]:
stem_basepoint, crown_basepoint = tree_utils.get_stem_endpoints(stem_cloud, ground_cloud)
print(f"Stem xy location is {np.round(stem_basepoint[:2],2)}")

**Stem Height**

In [ ]:
# estimate stem height
stem_height = crown_basepoint[2] - stem_basepoint[2]
print(f"Stem height is {stem_height:.2f}")

**Stem diameter at breastheight**

In [ ]:
breast_height = 1.3
dbh = tree_utils.diameter_at_breastheight(stem_cloud, stem_basepoint[2])
print(f"Stem diameter at breast height is {dbh:.2f} m")

**Breast cylinder fit**

In [ ]:
# Breast fit
stem_pts = np.asarray(stem_cloud.points)
breast_mask = np.where(stem_pts[:,2] < stem_basepoint[2] + breast_height)[0]
breast_cloud = stem_cloud.select_by_index(breast_mask)
breast_pts = np.array(breast_cloud.points)
cyl_center, cyl_axis, cyl_radius = fit_vertical_cylinder_3D(breast_pts, .05)[:3]
breast_cylinder = trimesh.creation.cylinder(radius=cyl_radius, sections=20, 
                        segment=(cyl_center-cyl_axis*breast_height/2, cyl_center+cyl_axis*breast_height/2)).as_open3d

o3d_utils.plot_mesh_cloud(breast_cylinder, breast_cloud)